<a href="https://colab.research.google.com/github/Shreyash007/CS6910-Deep-Learning-Course/blob/main/Assignment1(all_experiments).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

In [10]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 

Error in callback <function _WandbInit._resume_backend at 0x7f260b0cf1f0> (for pre_run_cell):


BrokenPipeError: ignored

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Error in callback <function _WandbInit._pause_backend at 0x7f260b0cf160> (for post_run_cell):


BrokenPipeError: ignored

#**Libraries used**
1. numpy: For matrix calculations
2. keras: For importing required dataset
3. wandb: For logging the data
4. tqdm: to create progress bar for loops
5. sklearn: For random test-train split 

In [2]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split
import wandb
from tqdm import tqdm

# **Q1 Solution**
Labelling dataset

In [3]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
#importing dataset(add this in show_images function as variable)

#checking size of training and test dataset
print("Training dataset shape, X=",X_train.shape,", Y=",Y_train.shape)
print("Test dataset shape, X=",X_test.shape,", Y=",Y_test.shape)

n_img_classes=len(np.unique(Y_train)) #to find out number of unique images 
img_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation


def show_images(n_img_classes,img_labels,X,Y):

   setImgs = []
   setLabels=[]

   for i in range(len(X)):
     if len(setImgs)==n_img_classes:
       break
     if img_labels[Y[i]] not in setLabels:
      setImgs.append(X[i])
      setLabels.append(img_labels[Y[i]])

   run=wandb.init(project='CS-6910 A1',entity='shreyashgadgil007',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(setImgs,setLabels))})
   run.finish()
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#--------------------------------------------------------------------------Q1 SOLUTION---------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#UNCOMMENT BELOW FUNCTION TO GET Q1 SOLUTION
#show_images(n_img_classes,img_labels,X_test,Y_test)


4422102/4422102 [==============================] - 0s 0us/step
Training dataset shape, X= (60000, 28, 28) , Y= (60000,)
Test dataset shape, X= (10000, 28, 28) , Y= (10000,)


# **Preprocessing dataset for operations**
1. Creating random training,test,validation split for dataset
2. Reshaping image matrix and normalizing pixel values
3. One hot encoding classifications


In [4]:
def dataset_preprocess():
  #change the code below to accept different dataset
  (X_1, Y_1), (X_test, Y_test) = fashion_mnist.load_data()
  
  #importing dataset again and normalizing
  X_1 = X_1.reshape(X_1.shape[0],-1)/255.0
  X_test = X_test.reshape(X_test.shape[0],-1)/255.0

  #training and validation split as specified in the question 10%
  X_train, X_val, Y_train, Y_val= train_test_split(X_1,Y_1,test_size=0.1,random_state=0)
  
  #one hot encoding
  Y_train_encoded=np.zeros((Y_train.shape[0],10))
  for i in range(len(Y_train)):
    Y_train_encoded[i][Y_train[i]]=1

  Y_val_encoded=np.zeros((Y_val.shape[0],10))
  for i in range(len(Y_val)):
    Y_val_encoded[i][Y_val[i]]=1

  Y_test_encoded=np.zeros((Y_test.shape[0],10))
  for i in range(len(Y_test)):
    Y_test_encoded[i][Y_test[i]]=1

  return X_train.T,X_test.T,X_val.T,Y_train.T,Y_val.T,Y_test.T,Y_train_encoded.T,Y_val_encoded.T,Y_test_encoded.T

#Below variables will be used in the entire code
X_train,X_test,X_val,Y_train,Y_val,Y_test,Y_train_encoded,Y_val_encoded,Y_test_encoded=dataset_preprocess()

print(Y_val_encoded.shape)
print(X_test.shape)
print(Y_train_encoded.shape)
print(X_train.shape)
print(Y_val.shape)
print(Y_test_encoded.shape)

(10, 6000)
(784, 10000)
(10, 54000)
(784, 54000)
(6000,)
(10, 10000)


# **Helper functions**
1. Declared activation functions and their gradients
2. Declared loss functions and their gradients
3. Declared initialisation functions 
4. Declared a function to initialize weights and biases

In [5]:
#---------------------------------------------------------------ACTIVATION FUNCTIONS AND THEIR GRADIENTS-------------------------------------------------------------------------------
def relu(X):
  return np.maximum(0,X)

def grad_relu(X):
  return X>0

def sigmoid(X):
  return 1/(1+np.exp(-X))

def grad_sigmoid(X):
  return (sigmoid(X))*(1-sigmoid(X))

def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def grad_tanh(X):
  return 1-(tanh(X))**2

def softmax(X):
  e_X = np.exp(X - np.max(X, axis = 0))
  return e_X / e_X.sum(axis = 0)


activation_function={
      'sigmoid':sigmoid,
      'relu':relu,
      'tanh':tanh
}

grad_activation_function={
      'sigmoid':grad_sigmoid,
      'relu':grad_relu,
      'tanh':grad_tanh
}
#---------------------------------------------------------------------------LOSS FUNCTION---------------------------------------------------------------------------------------------
def cross_entropy_loss(Y_encoded,Y,Y_pred,lambd,b):
 loss = (-np.sum(np.multiply(Y_encoded,np.log(Y_pred)))+((lambd/2.)*b))/Y.shape[0]
 return loss
 
def squared_error_loss(Y_encoded,Y,Y_pred,lambd,b):
  loss=((1/2)*(np.sum(Y_encoded-Y_pred))**2)/Y.shape[0]+(lambd*b*0)
  return loss

loss_function={
       'cross_entropy':cross_entropy_loss,
       'square_loss':squared_error_loss
}   


def grad_cross_entropy(Y,Y_pred):
  return (Y_pred-Y)

def grad_squared_error_loss(Y,Y_pred):  
  return (Y_pred-Y)*(Y_pred)*(1-Y_pred)

grad_loss_function={
    'cross_entropy':grad_cross_entropy,
    'square_loss':grad_squared_error_loss
}

#----------------------------------------------------------------------INITIALISATION FUNCTIONS---------------------------------------------------------------------------------------
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.3 #multiplied by 0.5 to have smaller values, to have better initialisation weights

def xavier_initialisation(shape):
    # Calculate the number of inputs and outputs
    n_in = shape[0]
    n_out = shape[1]    
    # Calculate the variance of the normal distribution
    variance = 2.0 / (n_in + n_out)
    # Initialize the weights with random values drawn from the normal distribution
    np.random.seed(0)
    weights = np.random.randn(n_in, n_out) * np.sqrt(variance)*2.0
    
    return weights

initialisation_function={
    'random':random_initialisation,
    'xavier':xavier_initialisation
}
#------------------------------------------------------------------INITIALIZING WEIGHTS AND BIASES------------------------------------------------------------------------------------


def initialize_w_b(input_layer,hidden_layer,output_layer,init):
  weights=[]
  biases=[]
  layers=[input_layer]+hidden_layer+[output_layer]
  for i in range(len(hidden_layer)+1): 
    weights.append(initialisation_function[init]((layers[i+1],layers[i])))
    biases.append(np.random.randn(layers[i+1],1)*0.3)
  return weights, biases

'''
#testing weights and biases output(for debugging shape size of matrix) 
output_layer = 10
input_layer = X_test.shape[1]
print(X_test.shape)
hidden_layer = [64, 64]
weights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'xavier')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)

weights2, biases2 = initialize_w_b(input_layer, hidden_layer, output_layer, 'random')
for i in range(len(weights2)):
    print(weights2[i].shape)
    
for i in range(len(biases2)):
    print(biases2[i].shape)
'''

"\n#testing weights and biases output(for debugging shape size of matrix) \noutput_layer = 10\ninput_layer = X_test.shape[1]\nprint(X_test.shape)\nhidden_layer = [64, 64]\nweights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'xavier')\nfor i in range(len(weights)):\n    print(weights[i].shape)\n    \nfor i in range(len(biases)):\n    print(biases[i].shape)\n\nweights2, biases2 = initialize_w_b(input_layer, hidden_layer, output_layer, 'random')\nfor i in range(len(weights2)):\n    print(weights2[i].shape)\n    \nfor i in range(len(biases2)):\n    print(biases2[i].shape)\n"

# **Neural Network**
1. With forward propagation
2. Backward propagation with different optimizers
3. A train function, which fits and trains the given set

# Q3 IMPLEMENTED BELOW

In [6]:
class NeuralNet():

  def __init__(self,input_layer,hidden_layer,output_layer,initialisation_func,act_function,loss_func,dropout_rate):
     self.input_layer=input_layer
     self.hidden_layer=hidden_layer
     self.output_layer=output_layer
     self.initialisation_func=initialisation_func
     self.act_function=act_function
     self.loss_func=loss_func
     self.weights,self.biases = initialize_w_b(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation_func)
     self.layer_size=len(self.hidden_layer)
     self.dropout_rate=dropout_rate


  
  def forward_propagation(self,X):
     #pre-activation
     self.a=[]
     #post-activation
     self.h=[]
     self.D=[]
     l=0
     
     #pre-activation and post-activation for input layer and first hidden layer
     self.a.append((self.weights[l]@X)+self.biases[l])#WX+b
     if(self.dropout_rate!=0):
                    dropRate=(1-self.dropout_rate)
                    d= np.random.rand(self.a[l].shape[0], self.a[l].shape[1])
                    d=d<dropRate
                    self.D.append(d)
                    self.a[l]=self.a[l]*d
                    self.a[l]=self.a[l]/dropRate
     self.h.append(activation_function[self.act_function](self.a[l]))
     
     #pre-activation and post-activation between hidden layers
     for l in range(1,self.layer_size):
       self.a.append((self.weights[l]@self.h[l-1])+self.biases[l])
       if(self.dropout_rate!=0):
                    dropRate=(1-self.dropout_rate)
                    d= np.random.rand(self.a[l].shape[0], self.a[l].shape[1])
                    d=d<dropRate
                    self.D.append(d)
                    self.a[l]=self.a[l]*d
                    self.a[l]=self.a[l]/dropRate
       self.h.append(activation_function[self.act_function](self.a[l]))
       
     #pre-activation and post-activation between last hidden layer and output layer
     l=self.layer_size 
     self.a.append((self.weights[l]@self.h[l-1])+self.biases[l])
     self.h.append(softmax(self.a[l]))
     
     return self.h[-1]

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------Q3 BACK PROPPAGATION FRAMEWORK WITH OPTIMIZATION FUNCTIONS-----------------------------------------------------------------
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
  def back_propagation(self,X,Y):

        g_a  = [0]*(self.layer_size+1)
        g_h  = [0]*(self.layer_size+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))
        batch_size = X.shape[1] 

        for k in reversed(range(self.layer_size+1)):
            #calculating loss function gradients for weights and biases at output
            if k == self.layer_size:
              g_a[k]=grad_loss_function[self.loss_func](Y,self.h[k])
            
            #calculating gradients for hidden layers     
            else:
                g_h[k] = (1/batch_size)*((self.weights[k+1].T)@(g_a[k+1]))
                g_a[k] = (1/batch_size)*((g_h[k])*(grad_activation_function[self.act_function](self.a[k])))#Here we use '*' operator for elementwise matrix multiplication
                if(self.dropout_rate!=0):
                    g_a[k]=g_a[k]* self.D[k-1]
                    g_a[k]=g_a[k]/(1-self.dropout_rate)
                
            #calculating gradients of weights 
            if k == 0:
                g_w[k] = (1/batch_size)*((g_a[k])@(X.T)) 
            else:
                g_w[k] = (1/batch_size)*((g_a[k])@(self.h[k-1].T))
            
            #calculating gradients of biases
            g_b[k]  = (1/batch_size)*np.sum(g_a[k], axis=1, keepdims = True)
        return g_w,g_b
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------STOCHASTIC GRADIENT DESCENT AND OTHER OPTIMIZERS-------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
  def stochastic_gradient_descent(self,W,b,g_w,g_b,lr,lambd):
      #Weights=Weights-(learning rate)*(grad weights)-(learning rate*lambda)*(Weights)(this is weight decay with L2 regularization)
      W=W-np.multiply(lr,g_w)-np.multiply(lr*lambd,W)

      #biases=biases-(learning rate)*(grad biases)
      b=b-np.multiply(lr,g_b)
      return W,b
  
  def momentum_based_gradient_descent(self,W,b,g_w,g_b, u_w_i, u_b_i, lr, lambd, gamma):
      #u_t=(beta)*(u_t-1)+(grad weight)[u_0 is initialised as zero]
      u_w=np.multiply(gamma,u_w_i)+g_w
      #weights=weights-(lr)*(updated_weights)-(weight decay L2 regularization)
      W=W-np.multiply(lr,u_w)-np.multiply(lr*lambd,W)
      
      #similarly for biases but without weight decay term
      u_b=np.multiply(gamma,u_b_i)+g_b
      b=b-np.multiply(lr,u_b)
      return W,b,u_w,u_b
  
  def nesterov_accelerated_gradient_descent(self,W,b,g_w,g_b,lr,lambd,gamma,u_w_i, u_b_i,b_input,b_output):
      w_t=W
      b_t=b
      #here we make changes to global variables as we need to update the weights(look ahead) for calculating gradients
      self.weights = self.weights - np.multiply(gamma,u_w_i)
      self.biases = self.biases - np.multiply(gamma,u_b_i)
      output =  self.forward_propagation(b_input)
      #in this step, gradients are recalculated in global variables with updated weight values 
      g_weights,g_biases = self.back_propagation(b_input,b_output)

      #u_t=(gamma)*(u_(t-1))+gradient of(w_t-(gamma)*(u_(t-1)))
      u_w=np.multiply(gamma,u_w_i)+g_weights
      #weights=weights-(lr)*(updated_weights)-(weight decay L2 regularization)
      w_t = w_t - u_w - np.multiply(lr*lambd,w_t)

      #similarly for biases but without weight decay term
      u_b=np.multiply(gamma,u_b_i)+g_b
      b=b-np.multiply(lr,u_b) 

      return w_t,b,u_w,u_b
  
  def rmsprop(self,W,b,g_w,g_b,lr,lambd,beta,eps,vw,vb):
      #V_t= (beta)*(V_t-1)+(1-beta)*(grad weight)^2 
      vw = np.multiply(vw,beta) + np.multiply(1-beta,np.power(g_w,2))           
      #similarly for biases
      vb = np.multiply(vb,beta) + np.multiply(1-beta,np.power(g_b,2))
      
      #w_t= (w_t-1)-((lr)/(V_t+eps)^(1/2))*(grad weight)-(L2 regularization weight decay)
      W = W - np.multiply(g_w,lr/np.power(vw+eps,1/2))- np.multiply(lr*lambd,W)
      #similarly for biases
      b = b - np.multiply(g_b,lr/np.power(vb+eps,1/2))
      return W,b,vw,vb
  
  def adam(self,W,b,g_w,g_b,beta1,beta2,lr ,m_t_i ,v_t_i ,m_b_i ,v_b_i,eps,i,lambd):
      
      m_t = np.multiply(beta1,m_t_i) + np.multiply(1-beta1,g_w)
      v_t = np.multiply(beta2,v_t_i) + np.multiply(1-beta2,np.power(g_w,2))
      m_b = np.multiply(beta1,m_b_i) + np.multiply(1-beta1,g_b)
      v_b = np.multiply(beta2,v_b_i) + np.multiply(1-beta2,np.power(g_b,2))
                
      #normalization of moment          
      m_hat_w = m_t/(1 - np.power(beta1,i+1))
      m_hat_b = m_b/(1 - np.power(beta1,i+1))

      #normalization          
      v_hat_w = v_t/(1 - np.power(beta2,i+1))
      v_hat_b = v_b/(1 - np.power(beta2,i+1))
      
      W = W - ((lr / np.power(v_hat_w + eps, 1/2)) * m_hat_w) - np.multiply(lr*lambd,W)
      
      b = b - ((lr / np.power(v_hat_b + eps, 1/2)) * m_hat_b)
      return W,b,m_t,v_t,m_b,v_b
  
  def nadam(self,W,b,g_w,g_b,beta1,beta2,lr ,m_t_i ,v_t_i ,m_b_i ,v_b_i,eps,i,lambd):
      m_t =  np.multiply(beta1,m_t_i) + np.multiply(1 - beta1,g_w)
      v_t =  np.multiply(beta2,v_t_i) + np.multiply(1 - beta2,np.power(g_w, 2))

      m_b =  np.multiply(beta1,m_b_i) + np.multiply(1 - beta1,g_b)
      v_b =  np.multiply(beta2,v_b_i) + np.multiply(1 - beta2,np.power(g_b, 2))
                
      m_hat_w = m_t / (1 - np.power(beta1, i+1)) 
      v_hat_t = v_t / (1 - np.power(beta2, i+1))

      m_hat_b = m_b / (1 - np.power(beta1, i+1)) 
      v_hat_b = v_b / (1 - np.power(beta2, i+1))
  
      a1 = (1-beta1)/(1-np.power(beta1,i+1))
      update_w = np.multiply(lr/(np.power(v_hat_t + eps,1/2)),(np.multiply(a1,g_w) + np.multiply(beta1,m_hat_w)))
      update_b = np.multiply(lr/(np.power(v_hat_b + eps,1/2)),(np.multiply(a1,g_b)+np.multiply(beta1,m_hat_b) ))
      W = W - update_w - np.multiply(lr*lambd,W)
      b = b - update_b    
      return W,b,m_t,v_t,m_b,v_b
  

  def predict(self, X,Y ):
      output =  self.forward_propagation(X)
      out_class=(np.argmax(output,axis=0))
      accuracy = round(self.accuracy_score(X, Y))
      return accuracy , out_class
  
  def accuracy_score(self, X, Y):
    pred_labels = np.argmax(self.forward_propagation(X), axis=0)
    return 100*(np.sum(pred_labels == Y) / len(Y))

  
  def predict_one_hot_encoded(self, X,Y ):
      output =  self.forward_propagation(X)
      accuracy = round(self.accuracy_score(X, Y))
      return output,accuracy


  def train(self,X_train,y_train,X_val ,y_val ,learning_rate,epochs, optimiser='gd',batch_size = 64,lambd=0.0005,WandB=False): 

      update_w = np.zeros(np.array(self.weights).shape)
      update_b = np.zeros(np.array(self.biases).shape)
      update_w_i = np.zeros(np.array(self.weights).shape)
      update_b_i = np.zeros(np.array(self.biases).shape)
      
      vw_i, vb_i, m_t_i, v_t_i, m_b_i, v_b_i=0.0,0.0,0.0,0.0,0.0,0.0
      m_t, v_t, m_hat_w, v_hat_w, m_b,v_b,m_hat_b,v_hat_b = 0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0 

      #below values taken from lecture slides for stability
      eps = 1e-8
      gamma = 0.9
      beta = 0.999
      beta1 = 0.9
      beta2 = 0.999
      train_accuracy, val_accuracy, training_loss ,validation_loss = [] ,[] ,[] ,[]
      
      
      for i in tqdm(range(epochs)):
        for batch in range(0, X_train.shape[1], batch_size):

          batch_images =  X_train[:,batch:batch+batch_size]
          batch_output =  Y_train_encoded[:,batch:batch+batch_size]
          output =  self.forward_propagation(batch_images)
          g_weights,g_biases = self.back_propagation(batch_images,batch_output)
          if optimiser == 'gd':
              self.weights,self.biases=self.stochastic_gradient_descent(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd)
          
          if optimiser == 'mgd':
              self.weights,self.biases,update_w,update_b=self.momentum_based_gradient_descent(self.weights,self.biases,g_weights,g_biases,update_w_i,update_b_i,learning_rate,lambd,gamma)
              update_w_i = update_w
              update_b_i = update_b

          if optimiser == 'ngd':
              w_t,b_t,update_w,update_b=self.nesterov_accelerated_gradient_descent(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd,gamma,update_w_i,update_b_i,batch_images,batch_output)
              self.weights = w_t
              self.biases = b_t
              update_w_i = update_w
              update_b_i = update_b

          if optimiser == 'rmsprop': 
              self.weights,self.biases,vw,vb= self.rmsprop(self.weights,self.biases,g_weights,g_biases,learning_rate,lambd,beta,eps,vw_i,vb_i)
              vw_i=vw
              vb_i=vb

          if optimiser == 'adam':
              self.weights,self.biases,m_t,v_t,m_b,v_b=self.adam(self.weights,self.biases,g_weights,g_biases,beta1,beta2,learning_rate, m_t_i, v_t_i, m_b_i, v_b_i,eps,i,lambd)
              m_t_i=m_t
              v_t_i=v_t
              m_b_i=m_b
              v_b_i=v_b
          
          if optimiser == 'nadam':
              self.weights,self.biases,m_t,v_t,m_b,v_b=self.nadam(self.weights,self.biases,g_weights,g_biases,beta1,beta2,learning_rate, m_t_i,v_t_i, m_b_i,v_b_i, eps, i, lambd)
              m_t_i=m_t
              v_t_i=v_t
              m_b_i=m_b
              v_b_i=v_b                         

        #Calculating accuracies 
        acc1=self.accuracy_score(X_train,y_train)
        train_accuracy.append(acc1)
  
        acc2=self.accuracy_score(X_val,y_val)
        val_accuracy.append(acc2)

        predicted_train = self.forward_propagation(X_train)
        predicted_val = self.forward_propagation(X_val)

        a =self.weights[1:len(self.hidden_layer)]
        b = np.sum([(np.sum((a[i]**2).reshape(1,-1))) for i in range(len(a))])#this is done to update loss function for weight decay problem 
        
        train_loss= loss_function[self.loss_func](Y_train_encoded, y_train, predicted_train, lambd,b )
        val_loss= loss_function[self.loss_func](Y_val_encoded, y_val, predicted_val, lambd,b )

        training_loss.append(train_loss)
        validation_loss.append(val_loss)
        #print(training_loss)
        #print(val_loss)

        print('Epoch {} : training_accuracy = {:.2f}, training_loss = {:.4f},Validation accuracy = {:.2f},Validation loss = {:.4f}'.format(i+1,acc1,train_loss, acc2,val_loss))
        if WandB:
          wandb.log({"val_accuracy": acc2,"accuracy": acc1,"steps":epochs,"train_loss":train_loss,"val_loss":val_loss},)

      
      return train_accuracy,val_accuracy,training_loss, validation_loss



A script for debugging and checking if all hyperparameters work properly.

In [9]:
n_class = 10

model = NeuralNet(28*28,[32,32,32],n_class, initialisation_func = 'xavier', act_function='sigmoid', loss_func = 'cross_entropy',dropout_rate=0.1)
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
acc1,acc2,train_loss, val_loss= model.train(X_train,Y_train,X_val ,Y_val ,learning_rate = 0.01,epochs=3, optimiser='nadam',batch_size =32,lambd=0.0005)
acc,Y_pred=model.predict(X_train,Y_train)

 33%|███▎      | 1/3 [00:04<00:09,  4.56s/it]

Epoch 1 : training_accuracy = 40.52, training_loss = 1.6392,Validation accuracy = 40.02,Validation loss = 1.6451


 67%|██████▋   | 2/3 [00:10<00:05,  5.64s/it]

Epoch 2 : training_accuracy = 45.84, training_loss = 1.4913,Validation accuracy = 45.60,Validation loss = 1.5081


100%|██████████| 3/3 [00:15<00:00,  5.14s/it]

Epoch 3 : training_accuracy = 47.69, training_loss = 1.4317,Validation accuracy = 48.17,Validation loss = 1.4445


# **Q2 SOLVED BELOW**
Forward propagation

In [ ]:
q1    = NeuralNet(X_train.shape[0],[64,64,64], 10,'xavier','sigmoid','cross_entropy')
A=q1.forward_propagation(X_train)
A=A.T
print(A[0])
print((Y_train_encoded.T)[0])

[0.0492696  0.72493824 0.0113433  0.01699098 0.00191964 0.00223488
 0.10496657 0.06102233 0.00334236 0.02397209]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


# **Sweep 1**
This sweep iterates over the hyperparameters, and the values are logged in wandb. I use **bayes** method, to iterate over hyperparameters as it decreases number of sweeps required. Bayesian optimization is computationally more expensive than random search, but it can be more efficient in terms of the number of iterations required to find the optimal set of hyperparameters.

Declaration: For the sweeps done below, I have followed below tutorial. 
https://theaisummer.com/weights-and-biases-tutorial/


In [7]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [10,20,30]
        },
        'learning_rate': {
            'values': [0.001,0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['random','xavier']
        },
        'batch_size':{
            'values':[32,64,128]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64],[128,128,128],[256,256,256],
                       [64,64,64,64],[64,64,64,64,64],
                       [128,128,128,128],[128,128,128,128,128]]
        },
        
        'weight_decay':{
            'values':[0.0,0.0005]
        },
        'dropout_rate':{
          'values':[0,0.1]  
        }
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'
      
  }

  # Initialize a new wandb run
  wandb.init(project='CS-6910 A1', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+'drop:'+str(wandb.config.dropout_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)

  
  config = wandb.config
  learning_rate = config.learning_rate
  epochs = int(config.epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initilisation
  dropout_rate=config.dropout_rate
  # Model training here
  sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function,dropout_rate)
  acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay,WandB=True)
  # print('Acc',acc2)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1, count=150)


Create sweep with ID: qt2i6eel
Sweep URL: https://wandb.ai/shreyashgadgil007/CS-6910%20A1/sweeps/qt2i6eel


wandb: Agent Starting Run: fb0zxg5d with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 128
wandb: 	dropout_rate: 0
wandb: 	epochs: 30
wandb: 	hidden_layer: [128, 128, 128]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.001
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: gd
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: shreyashgadgil007. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


  3%|▎         | 1/30 [00:06<03:08,  6.51s/it]

Epoch 1 : training_accuracy = 1.17, training_loss = 2.4815,Validation accuracy = 0.95,Validation loss = 2.4946


wandb: Ctrl + C detected. Stopping sweep.


# **Sweep2**
This is done to create a confusion matrix, by using hyperparameters which give highest validation accuracy  

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [20]
        },
        'learning_rate': {
            'values': [0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['xavier']
        },
        'batch_size':{
            'values':[64]
        },
        'optimiser': {
            'values': ['nadam']
        },
        'activation': {
            'values': ['tanh']
        },
        'hidden_layer': {
            'values': [[256,256,256]]
        },
        
        'weight_decay':{
            'values':[0.0005]
        }
        
    }
}
def get_confusion_matrix():
#change the values to Best Prediction from Above run
    config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'
    }

    wandb.init(config=config_defaults, magic=True)
    wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    config = wandb.config
    learning_rate = config.learning_rate
    epochs = int(config.epochs)
    hidden_layer = config.hidden_layer
    activation = config.activation
    optimiser = config.optimiser
    input_layer = config.input_layer
    output_layer = config.output_layer
    batch_size = config.batch_size
    weight_decay = config.weight_decay
    loss_function = config.loss_function
    initialisation = config.initilisation
    # Model training here
    sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function)
    acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay,WandB=True)
    acc,Y_pred=sweep_network.predict(X_test,Y_test)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                        y_true=Y_test, preds=Y_pred,
                        class_names=class_names)})
    
sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=get_confusion_matrix)


Create sweep with ID: dm8fk9h6
Sweep URL: https://wandb.ai/shreyashgadgil007/CS-6910%20A1/sweeps/dm8fk9h6


wandb: Agent Starting Run: bf6y2fx9 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	hidden_layer: [256, 256, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: shreyashgadgil007. Use `wandb login --relogin` to force relogin


  5%|▌         | 1/20 [00:34<10:48, 34.14s/it]

Epoch 1 : training_accuracy = 67.14, training_loss = 1.01560,Validation accuracy = 66.50,Validation loss = 1.02373


 10%|█         | 2/20 [01:04<09:34, 31.94s/it]

...

accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇██████
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▁▃▅▅▆▆▆▆▇▇▇▇▇▇▇█████
val_loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁
accuracy,77.96667
steps,20
train_loss,0.63199
val_accuracy,77.53333
val_loss,0.63761


wandb: Agent Starting Run: h3dd7evb with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	hidden_layer: [256, 256, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.0005


  5%|▌         | 1/20 [00:40<12:55, 40.82s/it]

Epoch 1 : training_accuracy = 67.14, training_loss = 1.01560,Validation accuracy = 66.50,Validation loss = 1.02373


wandb: Ctrl + C detected. Stopping sweep.


# **Sweep 3**
This sweep is done to compare squared error loss and cross-entropy loss for best model obtained. 

In [9]:

sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [20]
        },
        'learning_rate': {
            'values': [0.0001]
        },
        'loss_function':{
            'values':['cross_entropy','square_loss']
        },
        'initilisation':{
            'values':['xavier']
        },
        'batch_size':{
            'values':[32]
        },
        'optimiser': {
            'values': ['nadam']
        },
        'activation': {
            'values': ['tanh']
        },
        'hidden_layer': {
            'values': [
                       [256,256,256]]
        },
        'dropout_rate':{
          'values':[0]  
                },
        
        'weight_decay':{
            'values':[0.0005]
        }
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'
      
  }

  # Initialize a new wandb run
  wandb.init(project='CS-6910 A1', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)

  
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config
  learning_rate = config.learning_rate
  epochs = config.epochs
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initilisation
  dropout_rate = config.dropout_rate
  # Model training here
  sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function,dropout_rate)
  acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay,WandB=True)
  # print('Acc',acc2)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1, count=2)

Error in callback <function _WandbInit._resume_backend at 0x7f260b0cf1f0> (for pre_run_cell):


BrokenPipeError: ignored

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 439hilr7
Sweep URL: https://wandb.ai/shreyashgadgil007/CS-6910%20A1/sweeps/439hilr7


wandb: Agent Starting Run: jfj7m4sg with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0
wandb: 	epochs: 20
wandb: 	hidden_layer: [256, 256, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: square_loss
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.0005
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_init.py", line 1140, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_init.py", line 215, in setup
    tel.feature.set_init_tags = True
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/lib/telemetry.py", line 42, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 691, in _telemetry_callback
    self._telemetry_flush()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 702, in _telemetry_flush
    self._backend.i

Error in callback <function _WandbInit._pause_backend at 0x7f260b0cf160> (for post_run_cell):


BrokenPipeError: ignored

wandb: ERROR Abnormal program exit
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_init.py", line 1140, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_init.py", line 215, in setup
    tel.feature.set_init_tags = True
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/lib/telemetry.py", line 42, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 691, in _telemetry_callback
    self._telemetry_flush()
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 702, in _telemetry_flush
    self._backend.interface._publish_telemetry(self._telemetry_obj)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/interface/interface_shared.py", line 101, in _publish_telemetry
    self._publish(rec)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/interface/i

# **Preparing MNIST dataset**


In [ ]:
from keras.datasets import mnist
def dataset_preprocess():
  #change the code below to accept different dataset
  (X_1, Y_1), (X_test, Y_test) = mnist.load_data()
  
  #importing dataset again and normalizing
  X_1 = X_1.reshape(X_1.shape[0],-1)/255.0
  X_test = X_test.reshape(X_test.shape[0],-1)/255.0

  #training and validation split as specified in the question 10%
  X_train, X_val, Y_train, Y_val= train_test_split(X_1,Y_1,test_size=0.1,random_state=0)
  
  #one hot encoding
  Y_train_encoded=np.zeros((Y_train.shape[0],10))
  for i in range(len(Y_train)):
    Y_train_encoded[i][Y_train[i]]=1

  Y_val_encoded=np.zeros((Y_val.shape[0],10))
  for i in range(len(Y_val)):
    Y_val_encoded[i][Y_val[i]]=1

  Y_test_encoded=np.zeros((Y_test.shape[0],10))
  for i in range(len(Y_test)):
    Y_test_encoded[i][Y_test[i]]=1

  return X_train.T,X_test.T,X_val.T,Y_train.T,Y_val.T,Y_test.T,Y_train_encoded.T,Y_val_encoded.T,Y_test_encoded.T

#Below variables will be used in the entire code
X_train,X_test,X_val,Y_train,Y_val,Y_test,Y_train_encoded,Y_val_encoded,Y_test_encoded=dataset_preprocess()
print(Y_val_encoded[0])
print(X_test.shape)
print(Y_train_encoded.shape)
print(X_train.shape)
print(Y_test_encoded.shape)


11490434/11490434 [==============================] - 0s 0us/step
[0. 0. 0. ... 0. 0. 0.]
(784, 10000)
(10, 54000)
(784, 54000)
(10, 10000)


# **Sweep 4**
Done to find out accuracy

In [ ]:
sweep_config = {
    'method': 'grid', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [20]
        },
        'learning_rate': {
            'values': [0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['xavier']
        },
        'batch_size':{
            'values':[64]
        },
        'optimiser': {
            'values': ['nadam','adam','rmsprop']
        },
        'activation': {
            'values': ['tanh']
        },
        'hidden_layer': {
            'values': [
                       [256,256,256]]
        },
        
        'weight_decay':{
            'values':[0.0005]
        }
        
    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'
      
  }

  # Initialize a new wandb run
  wandb.init(project='CS-6910 A1', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)

  
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config
  learning_rate = config.learning_rate
  epochs = int(config.epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initilisation
  # Model training here
  sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function)
  acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay,WandB=True)
  # print('Acc',acc2)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1, count=3)

Create sweep with ID: fy591u1q
Sweep URL: https://wandb.ai/shreyashgadgil007/CS-6910%20A1/sweeps/fy591u1q


wandb: Agent Starting Run: b0oiox5a with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 20
wandb: 	hidden_layer: [256, 256, 256]
wandb: 	initilisation: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_function: cross_entropy
wandb: 	optimiser: nadam
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


  5%|▌         | 1/20 [00:30<09:41, 30.60s/it]

Epoch 1 : training_accuracy = 67.99, training_loss = 1.07635,Validation accuracy = 68.52,Validation loss = 1.05971


 10%|█         | 2/20 [01:03<09:33, 31.87s/it]

...

 95%|█████████▌| 19/20 [07:41<00:24, 24.13s/it]

Epoch 19 : training_accuracy = 86.82, training_loss = 0.44153,Validation accuracy = 86.88,Validation loss = 0.43521


100%|██████████| 20/20 [08:06<00:00, 24.34s/it]

Epoch 20 : training_accuracy = 86.99, training_loss = 0.43593,Validation accuracy = 86.98,Validation loss = 0.42985


accuracy,▁▄▅▅▆▆▇▇▇▇▇▇▇███████
steps,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,█▆▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▄▅▅▆▆▇▇▇▇▇▇████████
val_loss,█▆▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
accuracy,86.98889
steps,20
train_loss,0.43593
val_accuracy,86.98333
val_loss,0.42985
